

---



#Parsing process

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import random

seed_urls = ['https://cyberleninka.ru/article/c/psychology'] + [f'https://cyberleninka.ru/article/c/psychology/{i}' for i in range(2,3116)]

base_url = 'https://cyberleninka.ru'



---



In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br, zstd'
}

Collecting links to all the articles from cyberleninka's pages:

In [ ]:
# page_links = []
counter = len(page_links) / 20

for url in seed_urls[2501:]:
  response = requests.get(url, headers=headers)
  if response.status_code == 200:


    print(counter)
    counter += 1

    html_soup = BeautifulSoup(response.content, 'html.parser')
    title_divs = html_soup.find_all('div', class_='title')

    links = []
    for title_div in title_divs:
      parent_a = title_div.find_parent('a')
      if parent_a:
        page_links.append(base_url + parent_a['href'])

  time.sleep(random.randrange(2, 5))

In [ ]:
with open('all_cyberlen_article_links.txt', 'w') as file:
  print(page_links, file=file)



---



Going into obtained links to get the necessary info - author, year of publishing, the text itself. That save the information into a dictionary

In [ ]:
import ast

file = open('/content/cyberlen_articles_till_id3033_inclusieve.txt', 'r') #previously parsed texts
articles_data = ast.literal_eval(file.read())

file = open('/content/all_cyberlen_article_links.txt', 'r')
page_links = ast.literal_eval(file.read())

In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 4.6 MB/s eta 0:00:00


In [ ]:
from pypdf import PdfReader
from io import BytesIO

Все базовые данные для их сохранения в словарь

In [ ]:
def match(text, alphabet=set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')):
  return not alphabet.isdisjoint(text.lower())

def match2(text, symbols=set('ïîäðîñòêàìè')):
  return not symbols.isdisjoint(text.lower())

In [ ]:
articles_data = []

for page_link in page_links[2237:3000]:

  response = requests.get(page_link, headers=headers)
  if response.status_code == 200:

    html_soup = BeautifulSoup(response.content, 'html.parser')

    author_prep = html_soup.find('h2', class_="right-title").find('span').text
    author = author_prep[author_prep.find("—") + 2:]

    year = html_soup.find('div', class_="label year").text

    title = html_soup.find('i', itemprop="headline").text

    link = page_link + '/pdf'
    response = requests.get(link)
    response.raise_for_status()

    pdf_file = BytesIO(response.content)

    reader = PdfReader(pdf_file)

    all_text = ''.join(page.extract_text() for page in reader.pages)

    if author:
      ind = (all_text).find(author.split()[0])
      if ind != -1:
        all_text = all_text[ind:]

    if all_text and match(all_text) and not match2(all_text):
      articles_data.append({'author': author, 'year': year, 'title': title, 'text': all_text})


  time.sleep(random.uniform(0.5, 1))

In [ ]:
with open('cyberlen_id1001_1249_inclusieve.txt', 'w') as file:
  print(articles_data, file=file)



---



# Preprocessing texts

In [ ]:
!pip install pymystem3

В силу объема языкового корпуса данные обрабатывались поочередно секциями, после чего были объединены. Пример обработки:

In [ ]:
from pymystem3 import Mystem
from string import punctuation
import re
import pandas as pd
from multiprocessing import Pool
import os

# df = pd.read_csv('df_cyberlen_full.csv')

regex_replace_newline = re.compile(r'([\.,!?—:;])\n')
regex_replace_uppercase = re.compile(r'\n([A-Z])')
regex_replace_lowercase = re.compile(r'\n([a-z])')
regex_upper_rus = re.compile(r'\n([А-Я])')
regex_replace_letters = re.compile(r' ([гдеёжзлмнпртфхцчшщэюГДЕЁЖЗЛМНПРТФХЦЧШЩЭЮ]) ')
regex_separate_letters_digits = re.compile(r"([а-яёА-ЯЁ])(\d)")
regex_separate_digits_letters = re.compile(r"(\d)([а-яёА-ЯЁ])")
regex_remove_space_before_ся = re.compile(r' (ся) ')
regex_remove_space_before_ыь = re.compile(r' ([ыь]) ')


def text_newline_preprocessing(text):
    text = text.replace(' -\n', '').replace('-\n', '').replace('\uf0b7', ' ') \
        .replace('\u202f', ' ').replace('\xa0', ' ').replace('\uf020', ' ').replace('\xad', '') \
        .replace('\x84', ' ')

    text = regex_replace_newline.sub(r'\1 ', text)
    text = regex_replace_uppercase.sub(r' \1', text)
    text = regex_replace_lowercase.sub(r' \1', text)
    text = regex_upper_rus.sub(r' \1', text)
    text = regex_replace_letters.sub(r' \1', text)
    text = regex_separate_letters_digits.sub(r"\1 \2", text)
    text = regex_separate_digits_letters.sub(r"\1 \2", text)
    text = regex_remove_space_before_ся.sub(r'\1', text)
    text = regex_remove_space_before_ыь.sub(r'\1', text)

    text = text.replace('-', '').replace('   ', ' ').replace('  ', ' ').replace('\n', '')
    return text


m = Mystem()


def lemmatize(text):
    text = ''.join([c for c in text if c not in punctuation])
    lemmas = m.lemmatize(text.lower())
    return ''.join(lemmas)


def process_row(row):
    text0 = text_newline_preprocessing(row['text'])
    lemmas = lemmatize(text0)
    return {'text0': text0, 'lemmas': lemmas}


with Pool(processes=os.cpu_count()) as pool:
    results = pool.map(process_row, df.to_dict('records'))

df_processed = pd.DataFrame(results)
df = df.join(df_processed)

df.to_csv('preprocessed_cyberlen_df.csv', index=False)

In [ ]:
df

,author,year,title,text,text0,lemmas
0,"Бугелова Татьяна, Бенедикова Мариа, Чупкова Ле...",2019,Кросс-культурное исследование отношения к труд...,"Бугелова Татьяна, \nдоктор психологии, доцент...","Бугелова Татьяна, доктор психологии, доцент, и...",бугелов татьяна доктор психология доцент инсти...
1,Соколова Наталья Сергеевна,2011,О влиянии качеств личности студента на его усп...,"Соколова Н.С., 2011Соколова Н.С. О влиянии кач...","Соколова Н.С., 2011 Соколова Н.С. О влиянии ка...",соколова нс 2011 соколова нс о влияние качеств...
2,"Сабиров Аскадула Галимзянович, Сабирова Лиля А...",2021,ЧЕЛОВЕК-КОНСЬЮМЕРИСТ: ПОЗИТИВНЫЕ И НЕГАТИВНЫЕ ...,"Сабиров А. Г., Сабирова Л. А. \nАннотация. Ц...","Сабиров А. Г., Сабирова Л. А. Аннотация. Цель ...",сабиров а г сабирова л а аннотация цель исслед...
3,Терюшкова Юлия Юрьевна,2018,Индивидуальное консультирование как форма псих...,Терюшкова Ю. Ю. Восприятие отношений в группе...,Терюшкова Ю. Ю. Восприятие отношений в группе ...,терюшкова ю ю восприятие отношение в группа ка...
4,"Водопьянова Наталия Евгеньевна, Чесноков Влади...",2016,Дискуссионные аспекты психодиагностики свойств...,Водопьянова Наталия Евгеньевна\nдоктор психоло...,Водопьянова Наталия Евгеньевнадоктор психологи...,водопьянов наталия евгеньевнадоктор психологич...
...,...,...,...,...,...,...
95,"А В. Миненко, М В. Селиверстов",2023,СОВРЕМЕННЫЕ АСПЕКТЫ СРАВНЕНИЯ УРОВНЯ ЖИЗНИ СЕЛ...,АСПЕКТЫ СРАВНЕНИЯ УРОВНЯ ЖИЗНИ СЕЛЬСКИХ И \nГ...,АСПЕКТЫ СРАВНЕНИЯ УРОВНЯ ЖИЗНИ СЕЛЬСКИХ И ГОРО...,аспект сравнение уровень жизнь сельский и горо...
96,"Коваль Светлана Яковлевна, Кузнецова Наталья К...",2018,Проективная диагностика психофизиологических з...,"Коваль Светлана Яковлевна , кандидат медицинск...","Коваль Светлана Яковлевна , кандидат медицинск...",коваль светлана яковлевна кандидат медицински...
97,Фельдман Инесса Леонидовна,2017,Я-концепция субъекта самопознания: теория и эм...,Фельдман Инесса Леонидовна — кандидат психоло...,Фельдман Инесса Леонидовна — кандидат психолог...,фельдман инесса леонидовна — кандидат психолог...
98,Шулева Елизавета Игоревна,2017,Психологический анализ особенностей системы от...,Шулева Елизавета Игоревна \n \nкандидат педаг...,Шулева Елизавета Игоревна кандидат педагогиче...,шулев елизавета игоревна кандидат педагогичес...


From the full dataset we are deleting duplicates and checking the number of articles obtained:

In [ ]:
df = df[~df.duplicated(subset='text', keep='first')]

In [ ]:
len(df)

51438

In [ ]:
del df['text']

In [ ]:
df.head()

,author,year,title,text0,lemmas
0,Кровицкая Инна Владимировна,2012,Эмоциональная зрелость школьников,Кровицкая (Москва ) Аннотация . Рассматривает...,кровицкий москва аннотация рассматриваться ...
1,"Сафронова Маргарита Викторовна, Гаврилова Елен...",2011,Значение статуса семьи для психического здоров...,"Сафронова , Е.В. Гаврилова (Новосибирск ) Анн...",сафронов есть гаврилов новосибирск аннотаци...
2,Филоненко Анна Леонидовна,2010,"Базисные убеждения, особенности состояния и пе...",Филоненко (Томск) Аннотация. Представлены резу...,филоненко томск аннотация представлять результ...
3,Николаева Ирина Александровна,2010,Новый метод исследования личностных ценностей....,Николаева (Курган) Аннотация. Основа метода – ...,николаев курган аннотация основа метод – анали...
4,"Денисова Валентина Анатольевна, Кравцова Натал...",2011,Я-функции личности женщины и стиль переживания...,"Денисова , Н.А. Кравцова (Владивосток ) Аннот...",денисов на кравцова владивосток аннотация ...


Saving for using df in the term-searching process

In [ ]:
df.to_csv('preprocessed_cyberlen_df_no_text.csv', index=False)